In [1]:
import sys
sys.path.append("..")
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from tqdm import tqdm, trange
import random
import torch
import torch.nn.functional as F
from retrieval_model import BertEncoder
from torch import nn
import pandas as pd
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange
import pickle

In [2]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [3]:
# DPR model load
p_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/p_encoder")
q_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/q_encoder")
tokenizer =  AutoTokenizer.from_pretrained("klue/bert-base")

In [4]:
#train_dataset = load_from_disk("/opt/ml/data/train_dataset/train/")
#query_dataset = load_from_disk("../data/test_dataset/validation") # test query
#wiki_dataset = pd.read_json('../data/wikipedia_documents.json',orient='index') # wiki context
#wiki_dataset = pd.read_csv('/opt/ml/data/preprocess_wiki_doc.csv')
#train_dataset = load_from_disk("/opt/ml/data/train_dataset/new_validation/")
#origin_valid = load_from_disk("/opt/ml/data/train_dataset/validation/")

In [5]:
wiki_dataset = pd.read_csv('/opt/ml/data/preprocess_wiki_doc.csv')
query_dataset = load_from_disk("/opt/ml/data/train_dataset/validation/")

In [6]:
# num = 7
# print(train_dataset['question'][num])
# print(origin_valid['context'][num])
# print('---------------')
# print(train_dataset['context'][num])


In [7]:
#train_corpus = list(set([example for example in train_dataset['context']]))
wiki_corpus = list(set([example_wiki for example_wiki in wiki_dataset['text']]))
query = query_dataset['question']

In [8]:
print(len(query))
print(len(wiki_corpus))

240
55963


In [9]:
eval_batch_size = 32
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)
if torch.cuda.is_available():
    p_encoder.cuda()
    q_encoder.cuda()

# Construt dataloader
train_p_seqs = tokenizer(wiki_corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(
    train_p_seqs["input_ids"],
    train_p_seqs["attention_mask"],
    train_p_seqs["token_type_ids"]
)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(
    valid_dataset,
    sampler=valid_sampler,
    batch_size=eval_batch_size
)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

    epoch_iterator = tqdm(
        valid_dataloader,
        desc="Iteration",
        position=0,
        leave=True
    )
    p_encoder.eval()

    for _, batch in enumerate(epoch_iterator):
        batch = tuple(t.cuda() for t in batch)

        p_inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2]
        }
        
        outputs = p_encoder(**p_inputs).to("cpu").numpy()
        p_embs.extend(outputs)

torch.cuda.empty_cache()

Iteration: 100%|██████████| 1749/1749 [09:39<00:00,  3.02it/s]


In [16]:
train_q_seqs = tokenizer(
    query,
    max_length=64,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

query_dataset = TensorDataset(
    train_q_seqs["input_ids"],
    train_q_seqs["attention_mask"],
    train_q_seqs["token_type_ids"]
)

query_sampler = SequentialSampler(query_dataset)
query_dataloader = DataLoader(
    query_dataset,
    sampler=query_sampler,
    batch_size=eval_batch_size
)

q_embs = []

with torch.no_grad():

    epoch_iterator = tqdm(
        query_dataloader,
        desc="Iteration",
        position=0,
        leave=True
    )
    q_encoder.eval()

    for _, batch in enumerate(epoch_iterator):
        batch = tuple(t.cuda() for t in batch)

        q_inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2]
        }
        
        outputs = q_encoder(**q_inputs).to("cpu").numpy()
        q_embs.extend(outputs)

torch.cuda.empty_cache()
print('done')

Iteration: 100%|██████████| 8/8 [00:00<00:00, 26.88it/s]

done


In [17]:
p_embs = np.array(p_embs)
q_embs = np.array(q_embs)

print(p_embs.shape)
print(q_embs.shape)

(55963, 768)
(240, 768)


In [18]:
if torch.cuda.is_available():
    p_embs_cuda = torch.Tensor(p_embs).to('cuda')
    q_embs_cuda = torch.Tensor(q_embs).to('cuda')

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))
rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

In [19]:
dense_p_retrieval_result = {}
idx = 0
for i in tqdm(range(len(query))):
    p_list = []
    q = query[i]
    for j in range(100):
        p_list.append(wiki_corpus[rank[idx][j]])
    dense_p_retrieval_result[q] = p_list
    idx += 1

100%|██████████| 240/240 [00:00<00:00, 484.03it/s]


In [20]:
with open("/opt/ml/data/dense_valid_retrieval.bin", "wb") as file:
    pickle.dump(dense_p_retrieval_result,file)


# dense_n_retrieval_result = {}
# idx = 0
# for i in tqdm(range(len(query))):
#     p_list = []
#     q = query[i]
#     for j in range(10000,10004):
#         p_list.append(wiki_corpus[rank[idx][j]])
#     dense_n_retrieval_result[q] = p_list
#     idx += 1

In [21]:
idx = 14
print(query[idx])
dense_p_retrieval_result[query[idx]][:5]

반대동맹이 공산당과 갈라서겠다고 얘기한 날은 언제인가?


['반대동맹이 기동대의 위력을 목격한 1967년은 70년 안보로 이어지는 일본의 학생운동 절정기와 겹쳐 있었다. 혁신정당의 당리당략질에 절망하면서 국가권력에 대항할 방법을 모색하던 반대동맹은 측량말뚝 저지투쟁과 같은 시기에 발생한 하네다 사건 때 기동대와 대결한 신좌파 학생운동권에 기대를 걸고 “지원단체는 당파를 불문하고 받는다”는 자세를 취했다.group=주해|하네다 사건에 반대동맹 청년행동대 여러 명이 참여해 신좌파 학생들의 실력투쟁을 목격하고 왔다 한편 반국가권력투쟁을 내건 신좌파 각 정파에 있어서도 산리즈카 투쟁은 베트남전쟁 반전운동과group=주해|신좌파는 신도쿄 국제공항을 “일제의 해외침략기지”・“군사공항”으로 규정하고 투쟁대상으로 삼았다. 개항 이후의 사용 양태를 생각해 보면 엉뚱한 소리로 들리지만, 베트남전쟁 당시에는 하네다공항에 다수의 미군 전세기가 날아와 군사사용의 부정을 포함한 각각의 주장을 확인하는 각서를 교환한 적이 있고, 현재에도 연료부족을 이유로 미군 전세기가 임시착륙했을 때 상세한 명시와 군사미사용 재인을 항공회사가 요구하는 등 강한 감시하에 있다. 또한 반대동맹 키타하라파는 지금도 나리타 제3활주로 건설 움직임을 군사전용을 위한 것이라면서 집회참가를 호소하고 있다. 사토 내각에 대한 반발의 상징적 대상으로 보았기에, 쌍방의 기대가 일치했다 이듬해인 1968년 2월 26일 및 3월 10일, 나리타 시내에서 공항공단 분실에 돌입을 도모한 학생집단과 기동대 사이에 격렬한 충돌이 벌어졌다(나리타 데모 사건). 그전까지 신좌파에게 회의적이거나 폭력행사를 우려하던 반대파 주민들은 적극항쟁을 발휘한 학생들을 집에 초대해 환대했다. 2월 26일 충돌에서 반대동맹 대표 토무라 잇사쿠가 기동대에 구타당해 부상당한 것을 계기로 반대동맹은 무장투쟁 노선의 신좌파 정파인 중핵파・공산동・사청동 해방파가 주도하는 삼파전학련의 전폭적 지원을 받게 되었다 현지에서는 “전학련의 건투를 빈다”는 현수막이 각지에 붙고, “학생들만 맞게 두다니” “나가 먼저 혈제(血